In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
#df = pd.read_csv('amharic_awngi_data.xlsx', encoding=encoding)
df=pd.read_excel("/content/drive/My Drive/Amharic-Awngi/Data/Total-Awngi-Amharic_2.xlsx")

df.rename(columns={"Awngi ":"Awngi"},inplace=True)
df["Awngi"]=df["Awngi"].fillna(df["Awngi"].mode()[0])
import re
from unicodedata import normalize

def clean_text(text):
    text = normalize('NFD', text.lower())
    #text = re.sub('[^A-Za-z ]+', '', text)
    return text
df3=df.copy()
df['Awngi'] = df['Awngi'].apply(lambda row: clean_text(row))
df['Amharic'] = df['Amharic'].apply(lambda row: clean_text(row))
am = df['Amharic']
aw = df['Awngi']

am_max_len = max(len(line.split()) for line in am)
aw_max_len = max(len(line.split()) for line in aw)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

aw_tokenizer = Tokenizer()
aw_tokenizer.fit_on_texts(aw)
aw_sequences = aw_tokenizer.texts_to_sequences(aw)
aw_x = pad_sequences(aw_sequences, maxlen=aw_max_len, padding='post')

am_tokenizer = Tokenizer()
am_tokenizer.fit_on_texts(am)
am_sequences = am_tokenizer.texts_to_sequences(am)
am_y = pad_sequences(am_sequences, maxlen=am_max_len, padding='post')
am_vocab_size = len(am_tokenizer.word_index) + 1
aw_vocab_size = len(aw_tokenizer.word_index) + 1

Mounted at /content/drive


In [3]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, Box, HBox, VBox, Dropdown
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, Box, HBox, VBox
from IPython.display import display
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from IPython.display import display, HTML

am_model = load_model('/content/drive/My Drive/Amharic-Awngi/Models/LSTM_translation_model_june22.h5')
aw_model = load_model('/content/drive/My Drive/Amharic-Awngi/Models/Awngi-Amharic-LSTM_translation_model_june22.h5')

def translate_text(text, model, aw_tokenizer, am_tokenizer, aw_max_len):
    sequence = aw_tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=aw_max_len, padding='post')
    prediction = model.predict(padded_sequence, verbose=0)[0]
    indexes = [np.argmax(idx) for idx in prediction]
    return am_tokenizer.sequences_to_texts([indexes])[0]

def Am_Awngi_translate_text(text, model, am_tokenizer, aw_tokenizer, am_max_len):
    sequence = am_tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=am_max_len, padding='post')
    prediction = model.predict(padded_sequence, verbose=0)[0]
    indexes = [np.argmax(idx) for idx in prediction]
    return aw_tokenizer.sequences_to_texts([indexes])[0]

def translate_and_display(button):
    input_text = source_text_area.value
    if language_dropdown.value == 'Amharic to Awngi':
        translated_text = Am_Awngi_translate_text(input_text, am_model, am_tokenizer, aw_tokenizer, am_max_len)
    else:
        translated_text = translate_text(input_text, aw_model, aw_tokenizer, am_tokenizer, aw_max_len)
    translated_text_area.value = translated_text

def clear_output(button):
    source_text_area.value = ""
    translated_text_area.value = ""

source_text_area = widgets.Textarea(placeholder="Enter text to translate", rows=5, layout=widgets.Layout(width='50%', font_size='50px'))
translated_text_area = widgets.Textarea(placeholder="Translated text", rows=5, layout=widgets.Layout(width='50%', font_size='50px'))
translate_button = widgets.Button(description="Translate", button_style='primary', layout=widgets.Layout(width='auto', font_size='16px'))
clear_button = widgets.Button(description="Clear", button_style='danger', layout=widgets.Layout(width='auto', font_size='16px'))

translate_button.on_click(translate_and_display)
clear_button.on_click(clear_output)

language_dropdown = Dropdown(
    options=['Amharic to Awngi', 'Awngi to Amharic'],
    value='Amharic to Awngi',
    description='Translation Direction:',
    layout=widgets.Layout(width='auto', font_size='30px')
)

header_html = """
<div style="background-color: #2196F3; padding: 20px; text-align: center; color: white;">
  <h1 style="font-size: 32px;">Bidirectional Amharic to Awngi Language Translation System</h1>
</div>
"""

footer_html = """
<div style="background-color: #333; padding: 10px; text-align: center; color: white;">
  <p style="font-size: 16px;">Amharic To  Awngi Translation and Vice Versa</p>
</div>
"""

source_and_translated = HBox([source_text_area, translated_text_area], layout=widgets.Layout(padding='20px'))
controls = HBox([translate_button, clear_button, language_dropdown], layout=widgets.Layout(padding='20px'))

display(HTML(header_html), source_and_translated, controls, HTML(footer_html))